In [1]:
import pandas as pd
import string
import pandas as pd
import jellyfish
import random

# Abbreviation-Expansion List
Before we go into the details of the ILLOD tool, we first will give some insights into our evaluation data for AEP-Detection

In [2]:
data = pd.read_csv('abbr_db.CSV', names=['abbr', 'long_forms'], sep=';', encoding='utf8')
abbreviations = list(data['abbr'].values)
expansions = list(data['long_forms'].values)
#for i, abb in enumerate(abbreviations):
#    print(str(i) + ": " + abb + "| " + expansions[i])

# Introducing Helper Functions
These helper functions are needed in order to provide important methods for syntactic and semantic similarity measures and for ILLOD. We need a method to calculate the dice coefficient between two given strings since the jellyfish package doesnt provide this funcionality

In [3]:
def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    a_bigrams = set(a.lower())
    b_bigrams = set(b.lower())
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0 / (len(a_bigrams) + len(b_bigrams))

### Method to remove puntuation marks from  a given strings 

In [4]:
def clean_string(s):
    s_lower = s.lower()
    invalidcharacters = set(string.punctuation)
    if any(char in invalidcharacters for char in s):
        s_ = s_lower.translate(str.maketrans('', '', string.punctuation))
    else:
        s_ = s_lower
    return s_

### Method to remove stop words from  a given term

In [5]:
def stop_words_handling(term):
    splitted_term = term.split()
    stop_words = set(["for", "and", "of", "in", "via", "be"])
    
    if splitted_term[0] in stop_words:
        stop_words = stop_words - set([splitted_term[0]])
                
    for sw in stop_words:
        while sw in splitted_term:
            splitted_term.remove(sw)
    sanitized_term = " ".join([w for w in splitted_term]) 
        
    return sanitized_term

### Method to calculate and return $(a^{c}, potAbb(t^{c}))$  for a given pair $(a,t)$

In [6]:
def clean_string_pair_and_reduce_expansion(abb, term):
    abb_lower = abb.lower()
    term_lower = term.lower()
    sanitized_abbv = clean_string(abb_lower)
    sanitized_term = clean_string(term_lower)   
    sanitized_term_without_stopswords = stop_words_handling(sanitized_term)
    initial_letters_of_tokens_of_sanitized_term_without_stopswords = ''.join([c[0] for c in sanitized_term_without_stopswords.split()])
    return sanitized_abbv, initial_letters_of_tokens_of_sanitized_term_without_stopswords

# Classifiers based on semantic similarity (FastText)
## Algortihm 1

In [7]:
import fasttext
import fasttext.util
from scipy import spatial
# fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model("cc.en.300.bin")

In [8]:
def fast_text_similarity(a, t, threshold):
    
    a_v = ft.get_word_vector(a)
    t_v = ft.get_word_vector(t)
    if 1 - spatial.distance.cosine(a_v, t_v) >= threshold:
        return True
    else:
        return False

##  Cosine Similarity on Fasttext Wordvectors

In [9]:
def fast_text_sim(a, t):
    
    a_v = ft.get_word_vector(a)
    t_v = ft.get_word_vector(t)
    return 1 - spatial.distance.cosine(a_v, t_v)

# Classifiers based on syntactic similarity (LD, JWS, DC, DC)
## Algorithm 2 in different variants

In [10]:
def levensthein_distance_on_reduction_of_expansion(a, term, threshold):
    a_, t_ = clean_string_pair_and_reduce_expansion(a, term)
    if jellyfish.levenshtein_distance(a_, t_) <= threshold:
        return True
    else:
        return False

In [11]:
def jaro_winkler_similarity_on_reduction_of_expansion(a, term, threshold):
    a_, t_ = clean_string_pair_and_reduce_expansion(a, term)
    if jellyfish.jaro_winkler_similarity(a_, t_) >= threshold:
        return True
    else:
        return False

In [12]:
def dice_coefficient_on_reduction_of_expansion(a, term, threshold):
    a_, t_ = clean_string_pair_and_reduce_expansion(a, term)
    if dice_coefficient(a_, t_) >= threshold:
        return True
    else:
        return False

# ILLOD with its Methods (Section 4.3)

In [22]:
def check_initial_letters(a, t):
    initial_letters_of_tokens_of_t = ''.join([c[0] for c in t.split()])
    if initial_letters_of_tokens_of_t == a or initial_letters_of_tokens_of_t.upper() == a:
        return True

In [23]:
def check_length_consistency(a, t):
    length_consistency = False
    if len(t.split()) <= len(a):
        length_consistency = True
    return length_consistency

In [24]:
def check_order(a, t):
    abbv_reversed = a.lower()[::-1]
    term_reversed = t.lower()[::-1]
    len_of_term = len(t)
    
    pos_memory = 0
    pos_memory_list = []
    order_matching_string_rev = ""
    
    for j, char_from_abbv in enumerate(abbv_reversed):
        if j == len(abbv_reversed) - 1 and len(pos_memory_list) > 0 and pos_memory == len(term_reversed):
            break
        else:
            for i, char_from_term in enumerate(term_reversed[pos_memory:]):
                if char_from_abbv == char_from_term:
                    order_matching_string_rev = order_matching_string_rev + char_from_abbv
                    pos_memory = pos_memory + i + 1
                    pos_memory_list.append(len_of_term - pos_memory)
                    break
    if order_matching_string_rev == abbv_reversed:
        return True, pos_memory_list[::-1]
    else:
        return False, []

In [25]:
def check_distribution_of_matching_characters(pos_of_chars_list, t):
    term_intervals = []
    len_of_term = len(t)
    i = 0
    while i < len_of_term:
        sublist = []
        j = i
        while j < len_of_term and t[j] != " ":
            sublist.append(j)
            j = j+ 1
        i = j+1
        term_intervals.append(sublist)
        
    splitted_term = t.split()      
    
    containment_list = []
    for i, interval in enumerate(term_intervals):
        contanment_sublist = []
        for pos in pos_of_chars_list:
            if (pos in interval) and (splitted_term[i][0] == t[pos]):
                contanment_sublist.append(0)
            elif pos in interval:
                contanment_sublist.append(interval.index(pos))
        if len(contanment_sublist) == 0:
            contanment_sublist.append(-1)
        containment_list.append(contanment_sublist)
    
    result_of_distribution_check = False
    if len(containment_list) <= 1:
        result_of_distribution_check = True
    elif len (containment_list) >= 2:
        non_zero_count = 0
        for sublist in containment_list[1:]:
            if len(sublist) == 1 and 0 not in sublist:
                non_zero_count += 1
        if non_zero_count == 0:
            result_of_distribution_check = True
    
    return result_of_distribution_check

In [26]:
def illod(abbv, term, threshold=None):
    if (abbv[0].lower() == term[0].lower()):
        
        
        ###################################### Step (a) ##########################################
        # check wether initial letters of tokens in t match with the letters in abbreviation
        if check_initial_letters(abbv, term):
            return True
        
        
        
        ###################################### Step (b) ########################################
        # clean abbreviation and term from special characters and stopwords
        a_, t_ = clean_string_pair_and_reduce_expansion(abbv, term)
        if a_ == t_:
            return True
        
        sanitized_abbv = clean_string(abbv) 
        sanitized_term = clean_string(term)
        sanitized_term_without_stopswords = stop_words_handling(sanitized_term)
        sanitized_term_without_stopswords_splitted  = sanitized_term_without_stopswords.split()
        
        ###################################### Step (c), (d), (e) ###############################
        # Sequential call of the methods that check and compare lengths, order and distribution of characters
        length_consistency = check_length_consistency(sanitized_abbv, sanitized_term_without_stopswords)
        order, pos_of_chars_list = check_order(sanitized_abbv, sanitized_term_without_stopswords)
        distribution = check_distribution_of_matching_characters(pos_of_chars_list, sanitized_term_without_stopswords)


        if length_consistency and order and distribution:
            return True
        else:
            return False

        ################################## in case first letter differs ##########################
    else:
        return False

# Evaluation of the 3 different AEP-Detection Types (Section 4.4)

In [41]:
def find_and_count_false_negatives(algo, threshold):
    FN = 0
    for i, abb in enumerate(abbreviations):
        if not algo(abb, expansions[i], threshold):
            # print("\""+abb+"\""+", "+"\""+expansions[i]+"\"")
            FN += 1
    return FN, str(FN) + " FALSE NEGATIVES. Pairs that could not be detected out of " + str(len(abbreviations)) + " given pairs"

## New adjustment: The set $S$ is filled with AEPs, where both words have the  same initial letter

In [47]:
def find_and_count_false_positives(algo, threshold, alpha):
    il_dict = {}
    for abbr in abbreviations:
        iL = abbr[0].lower()
        if iL not in il_dict:
            il_dict[iL] = [exp for exp in expansions if exp[0].lower() == iL]
    
    test_set = []
    while len(test_set) <= alpha * len(abbreviations):
        rd1 = random.randint(0, len(abbreviations)-1)
        rd2 = random.randint(0, len(il_dict[abbreviations[rd1][0].lower()])-1)
        test_set.append((abbreviations[rd1], il_dict[abbreviations[rd1][0].lower()][rd2]))
            
    count_of_false_examples = 0
    FP = 0
    for j, tup in enumerate (test_set): 
        if algo(tup[0], tup[1], threshold):
            count_of_false_examples += 1
            FP +=1
    return FP, str(FP) + " FALSE POSITIVE detections out of " +  str(len(test_set)) + " created false examples"

## Test of new S creation method:

In [59]:
il_dict = {}
for abbr in abbreviations:
    iL = abbr[0].lower()
    if iL not in il_dict:
        il_dict[iL] = [exp for exp in expansions if exp[0].lower() == iL]

test_set = []
while len(test_set) <= 12:
    rd1 = random.randint(0, len(abbreviations)-1)
    rd2 = random.randint(0, len(il_dict[abbreviations[rd1][0].lower()])-1)
    test_set.append((abbreviations[rd1], il_dict[abbreviations[rd1][0].lower()][rd2]))

for tup in test_set:
    print(tup)

('OP', 'original poster')
('AVI', 'Amazon Web Services')
('FIFO', 'fast page mode')
('ADO', 'American Standard Code for Information Interchange')
('XSLT', 'X Window System')
('MRU', 'man-in-the-middle attack')
('RTC', 'Reverse Address Resolution Protocol')
('BIU', 'beginning of message')
('IPB', 'Intel comparative microprocessor performance')
('SD', 'System Management Mode')
('ASP', 'advanced power management')
('EMF', 'extended memory specification')
('GSM', 'gigabit / gigabyte')


## Test passed !

In [43]:
def determine_quality_parameters(alpha, algo, search_space_for_F1_optimization):
    max_f1 = 0
    best_values = []
    for th_ in search_space_for_F1_optimization:
        result_on_L = find_and_count_false_negatives(algo, th_)
        result_on_S = find_and_count_false_positives(algo, th_, alpha)
        FN = result_on_L[0]
        FP = result_on_S[0]
        TP = len(abbreviations) - FN
        
        # A classifier that does nothing is not useful. This serves to avoid a division by zero    
        if FP + TP == 0:
            precision = 0
            recall = 0
            f1 = 0
        else:
            precision = TP/(TP + FP)
            recall = TP/(TP + FN)
            f1 = (2*precision*recall)/(precision+recall)
            
        # memorise the best F1 value in the loop so far.       
        if f1 > max_f1:
            best_values = [th_, precision, recall, f1]
            max_f1 = f1
    return best_values

In [44]:
def evaluate_algorithm (algorithm, F1_optimization_search_space):
    eval_data = {}
    for alpha in [8, 16, 24, 48, 72]:
        max_f1 = 0
        best_values = []     
        eval_data[alpha] = determine_quality_parameters(alpha, algorithm, F1_optimization_search_space)
    return eval_data

In [45]:
step_list = [h/100 for h in list(range(0,100))]

## Evaluation metrics without filtering by initial letters (old values, we kept them for comparison with new terms)

In [73]:
pd.DataFrame.from_dict(evaluate_algorithm (fast_text_similarity, step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,threshold,precision,recall,F1
8,0.12,0.217585,0.437850,0.290706
16,0.13,0.128668,0.382979,0.192622
24,0.15,0.098640,0.288354,0.146996
48,0.16,0.053063,0.253639,0.087765
72,0.41,0.080103,0.052072,0.063115


In [74]:
# LD (LEVENSHTEIN_DISTANCE)
pd.DataFrame.from_dict(evaluate_algorithm (levensthein_distance_on_reduction_of_expansion, list(range(0, 4))), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,1,0.909033,0.805711,0.854259
16,1,0.831792,0.805711,0.818544
24,1,0.751436,0.805711,0.777628
48,0,0.989109,0.559351,0.714592
72,0,0.980373,0.559351,0.712299


In [75]:
# JWS (JARO-WINKLER-SIMILARITY)
pd.DataFrame.from_dict(evaluate_algorithm (jaro_winkler_similarity_on_reduction_of_expansion, step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,0.77,0.904564,0.854423,0.878779
16,0.79,0.869795,0.830347,0.849613
24,0.84,0.923861,0.760918,0.834510
48,0.84,0.862857,0.760918,0.808688
72,0.86,0.889685,0.695409,0.780641


In [76]:
# DC (DICE-COEFFICIENT)
pd.DataFrame.from_dict(evaluate_algorithm (dice_coefficient_on_reduction_of_expansion , step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,0.67,0.924566,0.775476,0.843484
16,0.76,0.872587,0.759239,0.811976
24,0.80,0.816265,0.758679,0.786419
48,0.82,0.871546,0.653415,0.746880
72,0.81,0.811544,0.653415,0.723945


In [77]:
# ILLOD
pd.DataFrame.from_dict(evaluate_algorithm (illod, [-1]), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,-1,0.986675,0.912094,0.947920
16,-1,0.969066,0.912094,0.939717
24,-1,0.961629,0.912094,0.936207
48,-1,0.927149,0.912094,0.919560
72,-1,0.890651,0.912094,0.901245


## New evaluation metrics with filtering by initial letters

In [49]:
pd.DataFrame.from_dict(evaluate_algorithm (fast_text_similarity, step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,threshold,precision,recall,F1
8,0.13,0.216799,0.382979,0.276867
16,0.13,0.119434,0.382979,0.182084
24,0.16,0.094829,0.253639,0.138047
48,0.16,0.049562,0.253639,0.082921
72,0.16,0.033294,0.253639,0.058862


In [48]:
# LD (LEVENSHTEIN_DISTANCE)
pd.DataFrame.from_dict(evaluate_algorithm (levensthein_distance_on_reduction_of_expansion, list(range(0, 4))), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,0,0.830424,0.559351,0.668451
16,0,0.731332,0.559351,0.633883
24,0,0.642444,0.559351,0.598025
48,0,0.487317,0.559351,0.520855
72,0,0.390692,0.559351,0.460051


In [50]:
# JWS (JARO-WINKLER-SIMILARITY)
pd.DataFrame.from_dict(evaluate_algorithm (jaro_winkler_similarity_on_reduction_of_expansion, step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,0.92,0.839181,0.642777,0.727964
16,0.92,0.689076,0.642777,0.665122
24,0.93,0.635369,0.617581,0.626349
48,0.94,0.477191,0.597424,0.530582
72,0.94,0.380257,0.597424,0.464721


In [51]:
# DC (DICE-COEFFICIENT)
pd.DataFrame.from_dict(evaluate_algorithm (dice_coefficient_on_reduction_of_expansion , step_list), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,0.84,0.787306,0.652856,0.713805
16,0.84,0.646341,0.652856,0.649582
24,0.92,0.603915,0.587346,0.595515
48,0.91,0.430271,0.587346,0.496686
72,0.86,0.325370,0.602464,0.422541


In [52]:
# ILLOD
pd.DataFrame.from_dict(evaluate_algorithm (illod, [-1]), orient="index", columns=["threshold", "precision", "recall", "F1"])

,threshold,precision,recall,F1
8,-1,0.720478,0.912094,0.805041
16,-1,0.575415,0.912094,0.705653
24,-1,0.466495,0.912094,0.617279
48,-1,0.309696,0.912094,0.462390
72,-1,0.228792,0.912094,0.365821
